In [53]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd

In [54]:
import pandas as pd

# List of CSV file paths
file_paths = [
    'Collected Dataset/idle.csv',
    'Collected Dataset/idle2.csv',
    'Collected Dataset/idle3.csv',
    'Collected Dataset/left_click.csv',
    'Collected Dataset/right_click.csv',
    'Collected Dataset/move_mouse.csv',
    'Collected Dataset/press_and_hold_left_click.csv',
    'Collected Dataset/scroll.csv'
]

# Read each CSV file into a list of DataFrames
csv_data = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
data = pd.concat(csv_data, ignore_index=True)

# Shuffle the DataFrame
data = data.sample(frac=1).reset_index(drop=True)

In [55]:
string_to_numeric = {'idle': 0, 'left_click': 1, 'right_click' : 2, 'move_mouse': 3, 'press_and_hold_left_click': 4, 'scroll': 5}
# data = (data - data.min()) / (data.max() - data.min())

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

y.replace(string_to_numeric, inplace=True)

X.sample(3)

C:\Users\ramez\AppData\Local\Temp\ipykernel_6976\3288166962.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace(string_to_numeric, inplace=True)


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
5124,0.600001,1.000000,0.796694,0.929101,0.841617,0.795034,0.697311,0.693244,0.503157,0.636832,...,0.238819,0.337298,0.274412,0.588804,0.163099,0.447969,0.078143,0.375736,0.000000,0.309346
12962,0.674071,1.000000,0.997710,0.843076,1.000000,0.637070,0.593688,0.523879,0.215825,0.492232,...,0.520585,0.640285,0.011533,0.648945,0.000000,0.510048,0.211194,0.582654,0.329668,0.667822
7162,1.000000,0.891962,0.875058,0.473753,0.622677,0.132183,0.314930,0.000000,0.046187,0.071774,...,0.504647,0.719205,0.215953,1.000000,0.000000,0.838364,0.213155,0.814916,0.361493,0.850887


In [56]:
num_states = len(y.unique())

X.shape, y.shape
num_states

6

In [57]:
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in a numpy array called 'landmarks_data' and labels in 'labels'
# landmarks_data.shape = (num_samples, num_landmarks, num_dimensions)
# labels.shape = (num_samples,)

# First, split your data into training and testing/validation sets
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Then, split the testing/validation set into separate testing and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Validation data shape:", X_val.shape)

# You can then use X_train, y_train for training, X_test, y_test for testing, and X_val, y_val for validation


Training data shape: (12880, 42)
Testing data shape: (1610, 42)
Validation data shape: (1610, 42)


In [58]:
# Define your TensorFlow model
model = models.Sequential([
    layers.Input(shape=(42,)),  # Input layer specifying the input shape
    tf.keras.layers.Dense(64, activation= 'relu'), 
    tf.keras.layers.Dense(128, activation= 'relu'), 
    tf.keras.layers.Dense(num_states, activation= 'softmax')
])
                                
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
# Train the model
model.fit(X_train, y_train, epochs = 10, validation_data=(X_val, y_val))

Epoch 1/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6175 - loss: 1.0769 - val_accuracy: 0.9174 - val_loss: 0.3261
Epoch 2/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9362 - loss: 0.2420 - val_accuracy: 0.9404 - val_loss: 0.2393
Epoch 3/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9530 - loss: 0.1724 - val_accuracy: 0.9491 - val_loss: 0.1940
Epoch 4/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9641 - loss: 0.1438 - val_accuracy: 0.9559 - val_loss: 0.1671
Epoch 5/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9711 - loss: 0.1164 - val_accuracy: 0.9559 - val_loss: 0.1582
Epoch 6/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9723 - loss: 0.1082 - val_accuracy: 0.9565 - val_loss: 0.1572
Epoch 7/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9728 - loss: 0.1049 - val_accuracy: 0.9596 - val_loss: 0.1521
Epoch 8/10
403/403 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9775 - loss: 0.0893 - val_accuracy: 0.

In [60]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9736 - loss: 0.1035
Test Loss: 0.09245144575834274
Test Accuracy: 0.9757763743400574


In [61]:
# output the model
model.save('../model/model2.keras')